In [ ]:
import collections
import heapq
import itertools
import os
import shutil
import string

import numpy as np
import pandas as pd

In [ ]:
class Problem(object):
    
    def __init__(self, problem_name, has_subtests=False):
        self.problem_name = problem_name
        self.test_id = 0
        self.has_subtests = has_subtests
        if os.path.exists(self.problem_name):
            shutil.rmtree(self.problem_name)
        os.makedirs(f'{self.problem_name}/input')
        os.makedirs(f'{self.problem_name}/output')
        
    def create_inputs(self, **kwargs):
        """Should return a list. Each element is a line in the input."""
        raise NotImplementedError
    
    def solve(self, inputs, **kwargs):
        """Should return a list. Each element is a line in the output."""
        raise NotImplementedError
    
    @staticmethod
    def save_to_file(data, file_path=None, **kwargs):
        if isinstance(data, list):
            data = ' '.join(str(x) for x in data)
        print(data, file=file_path, **kwargs)
    
    def create_single_test(self, inputs=None, **kwargs):
        n = 1
        if inputs is None:
            if self.has_subtests:
                if 'N' in kwargs:
                    n = kwargs['N']
                else:
                    n = np.random.randint(kwargs['N_MIN'], kwargs['N_MAX']+1)
                inputs = [n]
                outputs = []
                for _ in range(n):
                    inp = self.create_inputs(**kwargs)
                    outp = self.solve(inp)
                    inputs += inp
                    outputs += outp
            else:
                inputs = self.create_inputs(**kwargs)
                outputs = self.solve(inputs)
        else:
            outputs = self.solve(inputs, **kwargs)
            if self.has_subtests:
                inputs = [1] + inputs

        self.write_test(self.test_id, inputs, outputs)
        self.test_id += 1
        return (self.test_id-1, inputs, outputs)
    
    def create_multiple_tests(self, n, **kwargs):
        for i in range(n):
            yield self.create_single_test(**kwargs)
            
    def write_test(self, test_id, inputs, outputs):
        input_file = f'{self.problem_name}/input/input_{test_id:02d}.txt'
        output_file = f'{self.problem_name}/output/output_{test_id:02d}.txt'

        with open(input_file, 'w') as fp:
            for line in inputs:
                Problem.save_to_file(line, fp)

        with open(output_file, 'w') as fp:
            for line in outputs:
                Problem.save_to_file(line, fp)

In [ ]:
class TopCoderProblem(Problem):
    
    def parse_inputs(self, inputs):
        return inputs
    
    def parse_outputs(self, outputs):
        return outputs
    
    def create_from_string(self, str_data):
        data = str_data.split('\n')
        for line in data:
            line = line.strip()
            if not line:
                continue
            line = line.replace('{', '[').replace('}', ']')
            parts = [eval(part) for part in line.split('\t\t')[:-1]]
            assert len(parts) == 2
            
            inputs = self.parse_inputs(parts[0])
            outputs = self.parse_outputs(parts[1])
            
            self.write_test(self.test_id, inputs, outputs)
            self.test_id += 1
            yield (self.test_id - 1, inputs, outputs)

In [ ]:
class LotteryTicket(TopCoderProblem):
    def __init__(self):
        super().__init__('lottery-ticket')
    
    def parse_inputs(self, inputs):
        return inputs
    
    def parse_outputs(self, outputs):
        return [outputs]
        
prob = LotteryTicket()
for _ in prob.create_from_string(data):
    pass

In [ ]:
data = """	10, 1, 5, 10, 50		"POSSIBLE"		Passed	


	15, 1, 5, 10, 50		"POSSIBLE"		Passed	


	65, 1, 5, 10, 50		"POSSIBLE"		Passed	


	66, 1, 5, 10, 50		"POSSIBLE"		Passed	


	1000, 999, 998, 997, 996		"IMPOSSIBLE"		Passed	


	20, 5, 5, 5, 5		"POSSIBLE"		Passed	


	20, 5, 5, 5, 6		"IMPOSSIBLE"		Passed	


	400, 1000, 1000, 1000, 1000		"IMPOSSIBLE"		Passed	


	10, 2, 8, 5, 1		"POSSIBLE"		Passed	


	65, 25, 79, 59, 63		"IMPOSSIBLE"		Passed	


	962, 706, 146, 282, 828		"IMPOSSIBLE"		Passed	


	2, 2, 6, 3, 8		"POSSIBLE"		Passed	


	37, 37, 92, 5, 3		"POSSIBLE"		Passed	


	154, 154, 293, 383, 422		"POSSIBLE"		Passed	


	9, 7, 9, 6, 8		"POSSIBLE"		Passed	


	72, 27, 72, 39, 70		"POSSIBLE"		Passed	


	668, 913, 668, 300, 36		"POSSIBLE"		Passed	


	9, 5, 4, 2, 3		"POSSIBLE"		Passed	


	108, 34, 74, 65, 42		"POSSIBLE"		Passed	


	966, 712, 254, 869, 548		"POSSIBLE"		Passed	


	8, 5, 3, 8, 8		"POSSIBLE"		Passed	


	42, 60, 24, 42, 30		"POSSIBLE"		Passed	


	36, 779, 317, 36, 191		"POSSIBLE"		Passed	


	10, 3, 9, 7, 1		"POSSIBLE"		Passed	


	92, 43, 65, 49, 47		"POSSIBLE"		Passed	


	1536, 806, 891, 730, 371		"POSSIBLE"		Passed	


	9, 1, 7, 2, 4		"POSSIBLE"		Passed	


	54, 49, 30, 24, 85		"POSSIBLE"		Passed	


	1598, 955, 757, 841, 967		"POSSIBLE"		Passed	


	18, 7, 2, 9, 5		"POSSIBLE"		Passed	


	91, 40, 27, 24, 38		"POSSIBLE"		Passed	


	741, 539, 119, 83, 930		"POSSIBLE"		Passed	


	10, 2, 4, 6, 10		"POSSIBLE"		Passed	


	78, 59, 5, 31, 78		"POSSIBLE"		Passed	


	22, 307, 674, 387, 22		"POSSIBLE"		Passed	


	7, 6, 5, 3, 1		"POSSIBLE"		Passed	


	128, 30, 78, 74, 98		"POSSIBLE"		Passed	


	675, 513, 987, 291, 162		"POSSIBLE"		Passed	


	12, 7, 6, 8, 6		"POSSIBLE"		Passed	


	83, 75, 32, 53, 51		"POSSIBLE"		Passed	


	1909, 151, 942, 725, 967		"POSSIBLE"		Passed	


	17, 1, 8, 2, 8		"POSSIBLE"		Passed	


	150, 38, 58, 88, 54		"POSSIBLE"		Passed	


	1540, 384, 946, 910, 210		"POSSIBLE"		Passed	


	12, 9, 2, 9, 3		"POSSIBLE"		Passed	


	45, 47, 7, 31, 14		"POSSIBLE"		Passed	


	1355, 169, 901, 592, 763		"POSSIBLE"		Passed	


	21, 6, 1, 10, 5		"POSSIBLE"		Passed	


	218, 38, 49, 84, 96		"POSSIBLE"		Passed	


	685, 42, 603, 351, 292		"POSSIBLE"		Passed	


	13, 7, 5, 1, 7		"POSSIBLE"		Passed	


	218, 22, 49, 100, 69		"POSSIBLE"		Passed	


	1071, 485, 282, 735, 54		"POSSIBLE"		Passed	


	29, 10, 9, 9, 1		"POSSIBLE"		Passed	


	214, 89, 28, 68, 29		"POSSIBLE"		Passed	


	2835, 894, 649, 484, 808		"POSSIBLE"		Passed	


	4000, 1000, 1000, 1000, 1000		"POSSIBLE"		Passed	


	4000, 1000, 999, 1000, 1000		"IMPOSSIBLE"		Passed	


	2, 1, 5, 10, 50		"IMPOSSIBLE"		Passed	


	20, 42, 18, 35, 1		"IMPOSSIBLE"		Passed	


	65, 25, 79, 59, 63		"IMPOSSIBLE"		Passed	


	62, 6, 96, 32, 28		"IMPOSSIBLE"		Passed	


	37, 92, 196, 143, 28		"IMPOSSIBLE"		Passed	


	43, 142, 105, 153, 154		"IMPOSSIBLE"		Passed	


	96, 83, 22, 117, 219		"IMPOSSIBLE"		Passed	


	120, 198, 27, 72, 339		"IMPOSSIBLE"		Passed	


	295, 313, 68, 300, 236		"IMPOSSIBLE"		Passed	


	124, 354, 412, 273, 184		"IMPOSSIBLE"		Passed	


	369, 165, 142, 212, 254		"IMPOSSIBLE"		Passed	


	238, 248, 145, 213, 308		"IMPOSSIBLE"		Passed	


	179, 260, 324, 142, 530		"IMPOSSIBLE"		Passed	


	289, 617, 436, 91, 543		"IMPOSSIBLE"		Passed	


	249, 7, 641, 543, 365		"IMPOSSIBLE"		Passed	


	371, 447, 556, 141, 730		"IMPOSSIBLE"		Passed	


	349, 151, 607, 702, 394		"IMPOSSIBLE"		Passed	


	57, 80, 724, 285, 405		"IMPOSSIBLE"		Passed	


	209, 141, 467, 177, 432		"IMPOSSIBLE"		Passed	


	788, 795, 140, 877, 874		"IMPOSSIBLE"		Passed	


	542, 539, 119, 83, 930		"IMPOSSIBLE"		Passed	


	8, 1, 1, 1, 1		"IMPOSSIBLE"		Passed	


	7, 1, 1, 3, 4		"POSSIBLE"		Passed	


	30, 10, 5, 10, 10		"POSSIBLE"		Passed	


	200, 1, 2, 199, 872		"POSSIBLE"		Passed	


	10, 3, 5, 5, 6		"POSSIBLE"		Passed	


	1000, 250, 1, 500, 250		"POSSIBLE"		Passed	


	8, 2, 6, 7, 10		"POSSIBLE"		Passed	


	1, 1, 1, 1, 1		"POSSIBLE"		Passed	


	18, 1, 1, 9, 9		"POSSIBLE"		Passed	


	5, 10, 10, 2, 3		"POSSIBLE"		Passed	


	8, 100, 100, 100, 2		"IMPOSSIBLE"		Passed	


	3, 1, 1, 1, 7		"POSSIBLE"		Passed	


	11, 2, 5, 10, 1		"POSSIBLE"		Passed	


	6, 1, 3, 1, 3		"POSSIBLE"		Passed	


	10, 1, 2, 3, 4		"POSSIBLE"		Passed	


	20, 1, 1, 1, 2		"IMPOSSIBLE"		Passed	


	15, 1, 2, 3, 15		"POSSIBLE"		Passed	


	9, 3, 4, 5, 6		"POSSIBLE"		Passed	


	10, 2, 100, 12, 8		"POSSIBLE"		Passed	


	6, 2, 2, 2, 5		"POSSIBLE"		Passed	


	100, 25, 25, 101, 50		"POSSIBLE"		Passed	


	100, 10, 20, 50, 40		"POSSIBLE"		Passed	


	1004, 1, 2, 3, 1000		"POSSIBLE"		Passed	


	10, 1, 2, 30, 7		"POSSIBLE"		Passed	


	10, 5, 5, 2, 3		"POSSIBLE"		Passed	


	4, 1, 2, 5, 3		"POSSIBLE"		Passed	


	14, 1, 3, 5, 10		"POSSIBLE"		Passed	


	18, 5, 6, 1, 7		"POSSIBLE"		Passed	


	60, 30, 1000, 30, 1000		"POSSIBLE"		Passed	


	2, 1, 10, 1, 10		"POSSIBLE"		Passed	


	5, 2, 9, 9, 3		"POSSIBLE"		Passed	


	6, 2, 2, 3, 4		"POSSIBLE"		Passed	


	18, 18, 1, 1, 1		"POSSIBLE"		Passed	


	10, 5, 1, 1, 5		"POSSIBLE"		Passed	


	16, 1, 5, 12, 15		"POSSIBLE"		Passed	


	100, 30, 1, 20, 50		"POSSIBLE"		Passed	


	20, 10, 1, 1, 10		"POSSIBLE"		Passed	


	500, 200, 100, 300, 1		"POSSIBLE"		Passed	


	2, 1, 2, 3, 4		"POSSIBLE"		Passed	


	3, 1, 2, 5, 6		"POSSIBLE"		Passed	


	67, 30, 80, 37, 80		"POSSIBLE"		Passed	


	3, 1, 55, 1, 1		"POSSIBLE"		Passed	


	50, 1, 5, 10, 50		"POSSIBLE"		Passed	


	17, 2, 3, 5, 7		"POSSIBLE"		Passed	


	10, 5, 4, 3, 2		"POSSIBLE"		Passed	


	1000, 900, 5, 100, 800		"POSSIBLE"		Passed	


	100, 1, 2, 3, 100		"POSSIBLE"		Passed	


	5, 1, 3, 122, 1		"POSSIBLE"		Passed	


	3, 1, 100, 1, 1		"POSSIBLE"		Passed	


	10, 1, 4, 9, 100		"POSSIBLE"		Passed	


	4, 1, 2, 100, 200		"IMPOSSIBLE"		Passed	


	100, 1, 50, 1, 50		"POSSIBLE"		Passed	


	10, 2, 3, 8, 11		"POSSIBLE"		Passed	


	8, 1, 2, 3, 4		"POSSIBLE"		Passed	


	30, 1, 9, 11, 20		"POSSIBLE"		Passed	


	20, 1, 8, 2, 11		"POSSIBLE"		Passed	


	3, 1, 4, 1, 1		"POSSIBLE"		Passed	


	100, 1, 2, 1, 1		"IMPOSSIBLE"		Passed	


	50, 5, 10, 20, 25		"POSSIBLE"		Passed	


	100, 90, 1, 1, 10		"POSSIBLE"		Passed	


	6, 2, 1, 2, 2		"POSSIBLE"		Passed	


	5, 1, 2, 4, 100		"POSSIBLE"		Passed	


	11, 2, 5, 4, 10		"POSSIBLE"		Passed	


	17, 10, 3, 4, 5		"POSSIBLE"		Passed	


	400, 100, 100, 100, 400		"POSSIBLE"		Passed	


	5, 4, 3, 2, 2		"POSSIBLE"		Passed	


	11, 5, 6, 10, 50		"POSSIBLE"		Passed	


	300, 100, 1, 100, 100		"POSSIBLE"		Passed	


	3, 1, 1, 7, 1		"POSSIBLE"		Passed	


	100, 50, 1, 50, 1		"POSSIBLE"		Passed	


	20, 1, 10, 1, 1		"IMPOSSIBLE"		Passed	


	50, 20, 20, 30, 40		"POSSIBLE"		Passed	


	50, 45, 1, 25, 25		"POSSIBLE"		Passed	


	5, 1, 2, 4, 9		"POSSIBLE"		Passed	


	67, 30, 80, 27, 10		"POSSIBLE"		Passed	


	6, 2, 4, 100, 100		"POSSIBLE"		Passed	


	11, 1, 5, 10, 2		"POSSIBLE"		Passed	


	100, 25, 101, 25, 50		"POSSIBLE"		Passed	


	10, 1, 2, 20, 7		"POSSIBLE"		Passed	


	30, 10, 1, 10, 10		"POSSIBLE"		Passed	


	10, 10, 1, 2, 3		"POSSIBLE"		Passed	


	20, 10, 7, 6, 4		"POSSIBLE"		Passed	


	1011, 1, 10, 100, 1000		"POSSIBLE"		Passed	


	11, 1, 5, 10, 50		"POSSIBLE"		Passed	


	15, 5, 5, 5, 9		"POSSIBLE"		Passed	


	17, 10, 5, 4, 3		"POSSIBLE"		Passed	


	20, 2, 3, 1, 15		"POSSIBLE"		Passed	


	7, 1, 4, 6, 9		"POSSIBLE"		Passed	


	30, 10, 1, 2, 20		"POSSIBLE"		Passed	


	4, 1, 2, 3, 5		"POSSIBLE"		Passed	


	300, 100, 100, 1, 100		"POSSIBLE"		Passed	


	6, 1, 100, 2, 3		"POSSIBLE"		Passed	


	5, 2, 2, 3, 4		"POSSIBLE"		Passed	


	11, 1, 2, 4, 8		"POSSIBLE"		Passed	


	50, 15, 15, 60, 20		"POSSIBLE"		Passed	


	22, 5, 6, 10, 11		"POSSIBLE"		Passed	


	11, 5, 10, 2, 1		"POSSIBLE"		Passed	


	1000, 50, 300, 100, 700		"POSSIBLE"		Passed	


	7, 1, 3, 77, 3		"POSSIBLE"		Passed	


	1001, 1, 500, 400, 1000		"POSSIBLE"		Passed	


	13, 3, 7, 10, 15		"POSSIBLE"		Passed	


	20, 16, 12, 11, 8		"POSSIBLE"		Passed	


	3, 1, 1, 4, 1		"POSSIBLE"		Passed	


	1101, 1, 10, 100, 1000		"POSSIBLE"		Passed	


	10, 3, 4, 6, 9		"POSSIBLE"		Passed	


	3, 1, 10, 1, 1		"POSSIBLE"		Passed	


	100, 50, 25, 1, 25		"POSSIBLE"		Passed	


	31, 1, 10, 13, 20		"POSSIBLE"		Passed	


	20, 30, 10, 5, 5		"POSSIBLE"		Passed	


	10, 6, 50, 2, 2		"POSSIBLE"		Passed	


	25, 1, 29, 9, 15		"POSSIBLE"		Passed	


	10, 3, 1, 3, 4		"POSSIBLE"		Passed	


	2, 1, 100, 100, 100		"IMPOSSIBLE"		Passed"""